# Lab 4. Retail Operations Manager

## Introduction

In previous notebook we have shown you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. 

In this notebook we will interact with our supervisor agent created in the previous lab.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](img/energy_manager_agent.png)

On this lab you will execute a set of tests, using detailed trace, to check the chain-of-thought reasoning that is happening on the agents.

**[Chain-of-thought reasoning](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-prompt-engineering.html)** is a prompt engineering technique that give step-by-step reasoning on how an answer is derived based on your prompt.


With Amazon Bedrock Agents you can use [traces](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) to understand the step-by-step reasoning process of your agent. With the multi-agent collaboration feature, your supervisor trace also provides the reasoning details of its sub-agents

## Setup

Make sure that your boto3 version is the latest one.

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [ ]:
!pip freeze | grep boto3

## Invoking multi-agent Collaborator

Let's now invoke our supervisor agent with different queries to trigger the different sub-agents invocation. Before doing that, we're adding `bedrock_agent_helper.py` on Python path, so the file can be recognized and invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `invoke`: invoking an agent with a query

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

curr_month = datetime.now()

In [ ]:
import sys
import uuid
import time

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

Before continuing, let's recover the information from our supervisor agent and it's sub-agents from our environment.

In [ ]:
%store -r
print("Supervisor agent name:", retail_agent_name, "id:", retail_agent_id)
print("Sales agent name:", sales_agent_name, "id:", sales_agent_id, "alias_id:", sales_agent_alias_id)
print("Inventory agent name:", inventory_agent_name, "id:", inventory_agent_id, "alias_id:", inventory_agent_alias_id)
print("Store Operations agent name:", store_ops_agent_name, "id:", store_ops_agent_id, "alias_id:", store_ops_agent_alias_id)

We can now set a dictionary with agents names for better visualization of the traces

In [ ]:
multi_agent_names = {
    f"{sales_agent_id}/{sales_agent_alias_id}": sales_agent_name,
    f"{inventory_agent_id}/{inventory_agent_alias_id}": inventory_agent_name,
    f"{store_ops_agent_id}/{store_ops_agent_alias_id}": store_ops_agent_name,
    f"{retail_agent_id}/{retail_agent_alias_id}": retail_agent_name
}

On following section, you will invoke sub-agents but by multi-agent collaborator.

### Lab 1 - Sales Forecasting Agent
First let's query our agent with prompts requiring the sales forecasting agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted sales figures? How does it compare to my current sales? My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "can you give me my past sales data? What is my average sales volume in summer months? My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

future_2m = curr_month + relativedelta(months=2)
future_2m_formatted = future_2m.strftime("%Y/%m")

response = agents.invoke(
    f"Can you update my sales forecast for month {future_2m_formatted}? We have a major promotion planned and our estimate will be 25000. My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted sales month by month? My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 2 - Inventory Management Agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
response = agents.invoke(
    "How should I maintain inventory for Premium Series products in my store?", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can I get all tickets that I have? My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 3 - Store Operations Manager Agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "What departments have inefficient processes? My store id is 2.", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Where are my traffic peaks? How can I optimize my store operations? My store id is 1", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 4 - Multiple agents in parallel
Finally, let's submit a query which will result in our supervisor agent requiring responses from multiple agents in parallel. This shows the behaviour of a supervisor agent using supervisor mode rather than router mode as we have seen in the previous examples.

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you update my sales forecast for next month? We have a major promotion planned and our estimate will be 20000. My store id is 1. Then tell me how I should maintain inventory for Value Basics products in my store?", 
    retail_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

### Next Steps

Congratulations! You've completed the workshop! 

By now you have created 3 sub-agents and a supervisor agent. You have invoked the supervisor agent using prompts requiring multiple sub-agents.

For other multi-agent collaboration examples check the [Amazon Bedrock Agent Samples](https://github.com/awslabs/amazon-bedrock-agent-samples) repository 

Next, let's clean up our resources to avoid unexpected costs